# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [40]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
import time
from PIL import Image

In [41]:
# Implement a function for the validation pass
def validation(model, testloader, criterion):
    validation_loss = 0
    accuracy = 0
    for images, labels in testloader:

        # Move input and label tensors to the GPU
        images, labels = images.to(device), labels.to(device)

        output = model.forward(images)
        validation_loss += criterion(output, labels).item()

        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return validation_loss, accuracy

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [42]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [43]:
# TODO: Define your transforms for the training, validation, and testing sets
training_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])
testing_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
training_data = datasets.ImageFolder(data_dir + '/train', transform=training_transforms)

validating_data = datasets.ImageFolder(data_dir + '/valid', transform=testing_transforms)

testing_data = datasets.ImageFolder(data_dir + '/test', transform=testing_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)

validationloader = torch.utils.data.DataLoader(validating_data, batch_size=64)

testingloader = torch.utils.data.DataLoader(testing_data, batch_size=64)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [44]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [45]:
# TODO: Build and train your network
pre_trained_model = models.vgg16(pretrained = True)
#pre_trained_model

In [46]:
# Freeze parameters so we don't backprop through them
for param in pre_trained_model.parameters():
    param.requires_grad = False

#Create a new classifier
new_classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088,4096)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(4096, 400)),
                          ('relu', nn.ReLU()),
                          ('fc3', nn.Linear(400, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
pre_trained_model.classifier = new_classifier
#pre_trained_model

#set 'GPU Mode'
device = 'cuda'

# set 'CPU Mode', previously not used
#device = 'cpu'

criterion = nn.NLLLoss()
# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(pre_trained_model.classifier.parameters(), lr=0.001)

pre_trained_model.to(device)

for ii, (inputs, labels) in enumerate(trainloader):

    # Move input and label tensors to the GPU
    inputs, labels = inputs.to(device), labels.to(device)

    start = time.time()

    outputs = pre_trained_model.forward(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    if ii==5:
        break
        
#Calculated time train the new classifier
print('needed time for training the classifier: ', time.time() - start)


needed time for training the classifier:  0.6060123443603516


In [47]:
start = time.time()

#Train the pre_trained_model with the dataset, using 3 epochs and printing every 40 steps a intermediary result
epochs = 7
steps = 0
running_loss = 0
print_every = 20
for e in range(epochs):
    pre_trained_model.train()
    for images, labels in trainloader:
        steps += 1
        
        # Move input and label tensors to the GPU
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        output = pre_trained_model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            # Make sure network is in eval mode for inference
            pre_trained_model.eval()
            
            # Turn off gradients for validation, saves memory and computations
            with torch.no_grad():
                test_loss, accuracy = validation(pre_trained_model, validationloader, criterion)
                
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(validationloader)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(validationloader)))
            
            running_loss = 0
            
            # Make sure training is back on
            pre_trained_model.train()

print('training time for training the pre_trained_model with the dataset: ', time.time() - start)

Epoch: 1/7..  Training Loss: 6.017..  Test Loss: 3.199..  Test Accuracy: 0.285
Epoch: 1/7..  Training Loss: 2.980..  Test Loss: 2.200..  Test Accuracy: 0.450
Epoch: 1/7..  Training Loss: 2.091..  Test Loss: 1.471..  Test Accuracy: 0.617
Epoch: 1/7..  Training Loss: 1.801..  Test Loss: 1.234..  Test Accuracy: 0.643
Epoch: 1/7..  Training Loss: 1.557..  Test Loss: 0.892..  Test Accuracy: 0.747
Epoch: 2/7..  Training Loss: 1.283..  Test Loss: 0.917..  Test Accuracy: 0.753
Epoch: 2/7..  Training Loss: 1.209..  Test Loss: 0.810..  Test Accuracy: 0.780
Epoch: 2/7..  Training Loss: 1.085..  Test Loss: 0.782..  Test Accuracy: 0.786
Epoch: 2/7..  Training Loss: 1.040..  Test Loss: 0.659..  Test Accuracy: 0.817
Epoch: 2/7..  Training Loss: 1.048..  Test Loss: 0.643..  Test Accuracy: 0.833
Epoch: 3/7..  Training Loss: 0.862..  Test Loss: 0.644..  Test Accuracy: 0.825
Epoch: 3/7..  Training Loss: 0.949..  Test Loss: 0.590..  Test Accuracy: 0.843
Epoch: 3/7..  Training Loss: 0.796..  Test Loss: 0.5

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [48]:
# TODO: Do validation on the test set
correct = 0
total = 0
number_images = 0
with torch.no_grad():
    for data in testingloader:
        number_images += 1
        images, labels = data
        # Move input and label tensors to the GPU
        images, labels = images.to(device), labels.to(device)
        #outputs = pre_trained_model.forward(images)
        outputs = pre_trained_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #print('outputs: \n',outputs)
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    print('Number of Test images: ', number_images)

Accuracy of the network on the 10000 test images: 85 %
Number of Test images:  13


In [49]:
#Show the network
pre_trained_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [53]:
#define input and output size of the network
input_size = 3211264
output_size = 102
# TODO: Save the checkpoint
checkpoint = {'input_size': input_size,
              'outputsize': output_size,
              'hiddenlayers': [each.out_features for each in pre_trained_model.hidden],
              'state_dict': pre_trained_model.state_dict()}
torch.save(checkpoint, 'checkpoint.pth')

AttributeError: 'VGG' object has no attribute 'hidden'

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [9]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = fc_model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layers'])
    model.load_state_dict(checkpoint['state_dict'])
    
    return model
#load the state_dict(weights, etc.)in the network
pre_trained_model.load_state_dict(reloaded_state_dict)


In [10]:
trained_model = load_checkpoint('checkpoint.pth')
print(trained_model)

Reloded Model: 

 VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [11]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
   
    #Resize the image to 224 pixels
    image_resized = image.resize((256, 256))
    #crop the image to 224 pixels
    image_croped = image_resized.crop((0,0,224,224))
    #convert the image to a numpy array
    np_image = np.array(image_croped)
    #define mean and std deviation
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    #normalize the colors of the image
    np_image_normalized = (np_image - mean) / std
    #reorder the dimensions, because the color channel needs to be first
    np_image_transposed = np_image_normalized.transpose()
    #return np_image
    return np_image_transposed

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [12]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [34]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    #open image
    image = Image.open(image_path)
    # TODO: Implement the code to predict the class from an image file
    #images, labels = next(iter(testing_data))
    #img = images[0]
    imgage_processed = process_image(image)
    #convert the processed image to a torch tensor
    image_torch = torch.from_numpy(imgage_processed)
    #Set the model in inference mode
    model.eval()
    #Turn off autograd
    with torch.no_grad():
    # Calculate the class probabilities for img
        #output = model.forward(image_torch)
        output = pre_trained_model(image_torch)
    #ps = torch.exp(output)
    ##probs, classes = torch.topk(output, topk)
    
    ##return probs, classes
    return output

In [36]:
#for self test

filepath1 = 'flowers/test/1/image_06743.jpg'
image1 = Image.open(filepath1)
filepath2 = 'flowers/test/1/image_06752.jpg'
image2 = Image.open(filepath2)
filepath3 = 'flowers/test/1/image_06754.jpg'
image3 = Image.open(filepath3)
filepath4 = 'flowers/test/1/image_06760.jpg'
image4 = Image.open(filepath4)
filepath5 = 'flowers/test/1/image_06764.jpg'
image5 = Image.open(filepath5)

print('image1 format: ',image1.format )
print('image1 size: ',image1.size )
print('image1 mode: ',image1.mode )
print('image2 format: ',image2.format )
print('image2 size: ',image2.size )
print('image2 mode: ',image2.mode )
print('image3 format: ',image3.format )
print('image3 size: ',image3.size )
print('image3_mode: ',image3.mode )
print('image4 format: ',image4.format )
print('image4 size: ',image4.size )
print('image4 mode: ',image4.mode )
print('image5 format: ',image5.format )
print('image5 size: ',image5.size )
print('image5 mode: ',image5.mode )

image1 format:  JPEG
image1 size:  (500, 601)
image1 mode:  RGB
image2 format:  JPEG
image2 size:  (692, 500)
image2 mode:  RGB
image3 format:  JPEG
image3 size:  (593, 500)
image3_mode:  RGB
image4 format:  JPEG
image4 size:  (840, 500)
image4 mode:  RGB
image5 format:  JPEG
image5 size:  (667, 500)
image5 mode:  RGB


In [38]:
#For self test

image1_1 = process_image(image1)
image2_1 = process_image(image2)
image3_1 = process_image(image3)
image4_1 = process_image(image4)
image5_1 = process_image(image5)

#print('image1_1 format: ',image1_1.format )
#print('image1_1 size: ',image1_1.size )
#print('image1_1 mode: ',image1_1.mode )

print('\nimage1_1 dimensions: ', image1_1.shape)
print('image1_1 type: ', type(image1_1))
print('image1_1 are : ', image1_1.dtype)
print('\nimage2_1 dimensions: ', image2_1.shape)
print('image2_1 type: ', type(image2_1))
print('image2_1 are: ', image2_1.dtype)
print('\nimage3_1 dimensions: ', image3_1.shape)
print('image3_1 type: ', type(image3_1))
print('image3_1 are: ', image3_1.dtype)
print('\nimage4_1 dimensions: ', image4_1.shape)
print('image4_1 type: ', type(image4_1))
print('image4_1 are: ', image4_1.dtype)
print('\nimage5_1 dimensions: ', image5_1.shape)
print('image5_1 type: ', type(image5_1))
print('image5_1 are: ', image5_1.dtype)


image1_1 dimensions:  (3, 224, 224)
image1_1 type:  <class 'numpy.ndarray'>
image1_1 are :  float64

image2_1 dimensions:  (3, 224, 224)
image2_1 type:  <class 'numpy.ndarray'>
image2_1 are:  float64

image3_1 dimensions:  (3, 224, 224)
image3_1 type:  <class 'numpy.ndarray'>
image3_1 are:  float64

image4_1 dimensions:  (3, 224, 224)
image4_1 type:  <class 'numpy.ndarray'>
image4_1 are:  float64

image5_1 dimensions:  (3, 224, 224)
image5_1 type:  <class 'numpy.ndarray'>
image5_1 are:  float64


In [35]:
#Test the predict function
#probs, classes = predict(filepath1, pre_trained_model, topk=5)
output = predict(filepath1, pre_trained_model, topk=5)

RuntimeError: expected stride to be a single integer value or a list of 1 values to match the convolution dimensions, but got stride=[1, 1]

In [32]:
print('probs: ', probs)
print('classes: ',classes)

NameError: name 'probs' is not defined

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [16]:
# TODO: Display an image along with the top 5 classes